<a href="https://colab.research.google.com/github/aswinaus/LLM_Inference/blob/main/mistral_quant_awq_load_local.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install git+https://github.com/huggingface/transformers torch accelerate langchain langchain_huggingface

Code is essentially forcing Python to always use "UTF-8" as the preferred encoding, regardless of the user's actual system settings. UTF-8 is a widely used encoding that can represent a vast range of characters from different languages. By enforcing UTF-8, you can help ensure that your code works consistently across different platforms and avoids encoding-related errors. It's a common practice for improving compatibility and preventing issues with text handling in Python programs.

In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [3]:
from google.colab import userdata
HUGGING_FACE_TOKEN = userdata.get('HUGGING_FACE_TOKEN')

In [4]:
!huggingface-cli login --token $HUGGING_FACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: fineGrained).
The token `Agents` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `Agents`


In [5]:
from google.colab import drive
drive.mount('/content/drive')
# Download Data
data_dir = '/content/drive/MyDrive'

Mounted at /content/drive


In [6]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer
import transformers
import torch
from langchain_huggingface import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from threading import Thread

device = 'cuda' if torch.cuda.is_available() else 'cpu'

The nvidia-smi command is a utility provided by NVIDIA to query and display information about your NVIDIA GPU(s) (Graphics Processing Unit). This includes things like:

GPU model and name
Driver version
GPU utilization
Memory usage
Temperature
Power consumption
Processes running on the GPU

In [7]:
!nvidia-smi

Thu Feb  6 18:40:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             43W /  400W |       5MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

In [ ]:
!pip install autoawq
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

In [1]:
from typing import Tuple, Optional, Union, Dict, Any
from transformers import PreTrainedModel, AutoModel, AutoTokenizer, AutoConfig
from transformers.tokenization_utils_base import PreTrainedTokenizerBase

In [2]:
from google.colab import drive
drive.mount('/content/drive')
data_dir = '/content/drive/MyDrive' # Input a data dir path from your mounted Google Drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
quant_path = f"/{data_dir}/LLMs/Mistral/Mistral-Small-24B-Instruct-2501"

In [4]:
local_model_path = quant_path
local_tokenizer = AutoTokenizer.from_pretrained(quant_path)
local_model = AutoModel.from_pretrained(quant_path,low_cpu_mem_usage=True)

You have loaded an AWQ model on CPU and have a CUDA device available, make sure to set your model on a GPU device in order to run your model.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from datasets import load_dataset
import pandas as pd
dataset = load_dataset("aswinaus/tax_statistics_dataset_by_income_range", download_mode="force_redownload")
df=pd.DataFrame(dataset['train'])
df.head(10)

ModuleNotFoundError: No module named 'datasets'

In [2]:
from transformers import TrainingArguments, Trainer

# 1. Data Preparation (Example - adjust to your specific task)
def preprocess_function(examples):
    # Tokenize income ranges and tax statistics
    inputs = local_tokenizer(examples["income_range"], max_length=128, truncation=True)
    # ... further processing (e.g., converting tax statistics to numerical labels)
    return inputs

# Apply preprocessing to the DataFrame
df = df.map(preprocess_function, batched=True)

# 2. Model Adaptation (Fine-tuning example)
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,  # Adjust as needed
    # ... other training arguments
)

trainer = Trainer(
    model=local_model,
    args=training_args,
    train_dataset=df["train"],  # Assuming you have a train split
    eval_dataset=df["validation"], # Assuming you have a validation split
    # ... data collator, compute_metrics if needed
)

# 3. Training
trainer.train()

# 4. Evaluation (example)
eval_results = trainer.evaluate()
print(eval_results)

NameError: name 'df' is not defined